In [15]:
import cv2
import numpy as np

In [21]:
INPUT_FILE = 'ALA1934_RR.pdf-5_1.png'
INPUT_FILE_BASE = INPUT_FILE[:INPUT_FILE.rindex('.')]

img = cv2.imread(INPUT_FILE)
img_w, img_h = img.shape[:2]
img_w, img_h

(3505, 2480)

In [22]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray, 50, 150, apertureSize=3)
cv2.imwrite(INPUT_FILE_BASE + '-edges.png', edges)

True

In [ ]:
#![edges](ALA1934_RR.pdf-3_1-edges.png)

In [76]:
lines_only = np.zeros((img_w, img_h, 1), np.uint8)

max_img_dim = max(img_w, img_h)
min_line_len = max_img_dim / 50
max_line_gap = max_img_dim / 200

print('min_line_len', min_line_len)
print('max_line_gap', max_line_gap)

res_rho = 1
res_theta = np.pi/500
votes_thresh = 100
lines = cv2.HoughLinesP(edges, res_rho, res_theta, votes_thresh, min_line_len, max_line_gap)

print('len(lines)', len(lines))

for l in lines:
    l = l[0]
    p1, p2 = tuple(l[:2]), tuple(l[2:])
    cv2.line(lines_only, p1, p2, 255, 5)
    cv2.line(img, p1, p2, (0,255,0), 5)

cv2.imwrite(INPUT_FILE_BASE + '-linesonly.png', lines_only)
cv2.imwrite(INPUT_FILE_BASE + '-hough.png', img)

min_line_len 70.1
max_line_gap 17.525
len(lines) 703


True

In [20]:
### Lines
#![lines](ALA1934_RR.pdf-3_1-linesonly.png)

### Lines on orig. Image
#![hough](ALA1934_RR.pdf-3_1-hough.png)

In [77]:
from math import degrees, radians

from pdftabextract.geom import pt, vecangle

UNITY_VECS = (
    pt(0, 1),   # up
    pt(1, 0),   # right
    pt(0, -1),  # down
    pt(-1, 0)   # left
)


# find rotation / skew

# lines = (
#     (0, 0, 0, 1),   # up
#     (0, 1, 0, 0),   # down
#     (0, 0, 1, 0),   # right
#     (1, 0, 0, 0),   # left
#     (0, 0, 0.1, 1), # up2
#     (0.1, 1, 0, 0), # down2
#     (0, 0, -0.1, 1), # up3
#     (-0.1, 1, 0, 0), # down3
#     (0, 0, 1, 0.1),   # right2
#     (1, 0.1, 0, 0),   # left2
#     (0, 0, 1, -0.1),   # right3
#     (1, -0.1, 0, 0),   # left3
# )

for l in lines:
    l = l[0]
    print(l)
    l_vec = l[2:] - l[:2]
    
    alphas = np.array([vecangle(l_vec, unity) for unity in UNITY_VECS])
    print(list(map(degrees, alphas)))
    min_idx = np.argmin(alphas)
    min_val = alphas[min_idx]
    
    if min_idx in (2, 3):   # down or left -> negate!
        min_val *= -1
    
    print(degrees(min_val))
    print()

[1327 2302 1327 2226]
[180.0, 90.0, 0.0, 90.0]
-0.0

[1992 2721 1992 2653]
[180.0, 90.0, 0.0, 90.0]
-0.0

[1757 2170 1757 2134]
[180.0, 90.0, 0.0, 90.0]
-0.0

[1181 2645 1181 2621]
[180.0, 90.0, 0.0, 90.0]
-0.0

[1331 1356 1331 1292]
[180.0, 90.0, 0.0, 90.0]
-0.0

[ 251 1585  251 1547]
[180.0, 90.0, 0.0, 90.0]
-0.0

[1255 2191 1255 2134]
[180.0, 90.0, 0.0, 90.0]
-0.0

[1118 2643 1118 2561]
[180.0, 90.0, 0.0, 90.0]
-0.0

[1921 1101 1921 1045]
[180.0, 90.0, 0.0, 90.0]
-0.0

[1916 1940 1916 1879]
[180.0, 90.0, 0.0, 90.0]
-0.0

[1402  718 1402  681]
[180.0, 90.0, 0.0, 90.0]
-0.0

[1259 2189 1259 2134]
[180.0, 90.0, 0.0, 90.0]
-0.0

[2056 1067 2056 1014]
[180.0, 90.0, 0.0, 90.0]
-0.0

[2062 1006 2062  960]
[180.0, 90.0, 0.0, 90.0]
-0.0

[1986 1271 1986 1189]
[180.0, 90.0, 0.0, 90.0]
-0.0

[1398  633 1398  597]
[180.0, 90.0, 0.0, 90.0]
-0.0

[1181  759 1181  691]
[180.0, 90.0, 0.0, 90.0]
-0.0

[1112  940 1112  866]
[180.0, 90.0, 0.0, 90.0]
-0.0

[1177 3037 1255 3037]
[90.0, 0.0, 90.0, 180.0]